In [ ]:
pip install -U scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 20.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import ipywidgets as widgets
import matplotlib.pyplot as plt

# ----------------------
# Define fuzzy variables
# ----------------------
service = ctrl.Antecedent(np.arange(0, 11, 1), 'service')  # 0–10 scale
food = ctrl.Antecedent(np.arange(0, 11, 1), 'food')        # 0–10 scale
tip = ctrl.Consequent(np.arange(5, 26, 1), 'tip')          # 5–25% tip

# ----------------------
# Membership functions
# ----------------------
# Service
service['poor'] = fuzz.trapmf(service.universe, [0, 0, 2, 4])
service['good'] = fuzz.trimf(service.universe, [2, 5, 8])
service['excellent'] = fuzz.trapmf(service.universe, [6, 8, 10, 10])

# Food
food['bad'] = fuzz.trapmf(food.universe, [0, 0, 3, 7])
food['delicious'] = fuzz.trapmf(food.universe, [3, 7, 10, 10])

# Tip
tip['cheap'] = fuzz.trapmf(tip.universe, [5, 5, 10, 13])
tip['average'] = fuzz.trimf(tip.universe, [10, 15, 20])
tip['generous'] = fuzz.trapmf(tip.universe, [17, 20, 25, 25])

# ----------------------
# Define rules
# ----------------------
rule1 = ctrl.Rule(service['poor'] | food['bad'], tip['cheap'])
rule2 = ctrl.Rule(service['good'], tip['average'])
rule3 = ctrl.Rule(service['excellent'] | food['delicious'], tip['generous'])

# ----------------------
# Control system
# ----------------------
tipping_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
tipping = ctrl.ControlSystemSimulation(tipping_ctrl)

# ----------------------
# Interactive function
# ----------------------
def update_tip(srv, fd):
    tipping.input['service'] = srv
    tipping.input['food'] = fd
    tipping.compute()

    # Compute membership degrees for inputs
    srv_poor = fuzz.interp_membership(service.universe, service['poor'].mf, srv)
    srv_good = fuzz.interp_membership(service.universe, service['good'].mf, srv)
    srv_excellent = fuzz.interp_membership(service.universe, service['excellent'].mf, srv)

    fd_bad = fuzz.interp_membership(food.universe, food['bad'].mf, fd)
    fd_delicious = fuzz.interp_membership(food.universe, food['delicious'].mf, fd)

    # Print results
    print(f"Service = {srv}, Food = {fd} → Tip = {tipping.output['tip']:.2f}%")
    print(f"Membership Degrees:")
    print(f" Service → Poor: {srv_poor:.2f}, Good: {srv_good:.2f}, Excellent: {srv_excellent:.2f}")
    print(f" Food → Bad: {fd_bad:.2f}, Delicious: {fd_delicious:.2f}")

    # --- Plot fuzzification for service ---
    plt.figure(figsize=(6,4))
    plt.plot(service.universe, service['poor'].mf, 'b', label='Poor')
    plt.plot(service.universe, service['good'].mf, 'g', label='Good')
    plt.plot(service.universe, service['excellent'].mf, 'r', label='Excellent')
    plt.axvline(srv, color='k', linestyle='--', label=f'Input Service={srv}')
    plt.title("Service Membership Functions")
    plt.xlabel("Service Level")
    plt.ylabel("Membership Degree")
    plt.legend()
    plt.show()

    # --- Plot fuzzification for food ---
    plt.figure(figsize=(6,4))
    plt.plot(food.universe, food['bad'].mf, 'b', label='Bad')
    plt.plot(food.universe, food['delicious'].mf, 'r', label='Delicious')
    plt.axvline(fd, color='k', linestyle='--', label=f'Input Food={fd}')
    plt.title("Food Membership Functions")
    plt.xlabel("Food Quality")
    plt.ylabel("Membership Degree")
    plt.legend()
    plt.show()

    # --- Plot defuzzification for tip ---
    tip.view(sim=tipping)

# ----------------------
# Interactive sliders
# ----------------------
srv_slider = widgets.IntSlider(value=5, min=0, max=10, step=1, description='Service:')
fd_slider = widgets.IntSlider(value=5, min=0, max=10, step=1, description='Food:')

widgets.interact(update_tip, srv=srv_slider, fd=fd_slider)


interactive(children=(IntSlider(value=5, description='Service:', max=10), IntSlider(value=5, description='Food…

<function __main__.update_tip(srv, fd)>